## Data Ingestion

In [1]:
import pandas as pd

# column_names = ['id', 'text', 'label']
constraint_train = pd.read_csv("/kaggle/input/dataset-4/constraint_train.csv")
constraint_test = pd.read_csv("/kaggle/input/dataset-4/constraint_test.csv")
constraint_dev = pd.read_csv("/kaggle/input/dataset-4/constraint_dev.csv")

In [2]:
constraint_train.shape, constraint_test.shape, constraint_dev.shape

((4494, 4), (963, 4), (963, 4))

## Data Preparation

In [3]:
LABELS = constraint_train['label'].unique().tolist()
train_label = constraint_train['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_23/1124859992.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_label = constraint_train['label'].replace(LABELS, [0, 1]).tolist()


In [4]:
train_data = constraint_train['tweet'].tolist()

In [5]:
dev_data = constraint_dev['tweet'].tolist()
dev_label = constraint_dev['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_23/2472027419.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dev_label = constraint_dev['label'].replace(LABELS, [0, 1]).tolist()


In [6]:
test_data = constraint_test['tweet'].tolist()
test_label = constraint_test['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_23/1887347934.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_label = constraint_test['label'].replace(LABELS, [0, 1]).tolist()


In [7]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class ClassificationDataset(Dataset):
    def __init__(self, texts: list[str], labels: list[int], tokenizer, max_length: int):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text, 
            add_special_tokens=True, 
            max_length=self.max_length, 
            return_token_type_ids=False, 
            padding="max_length",
            truncation=True, 
            return_attention_mask=True, 
            return_tensors="pt"
        )
        
        return {
            "input_ids": encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [8]:
from transformers import RobertaTokenizer, RobertaModel, AdamW

max_length = 128

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_dataset = ClassificationDataset(train_data, train_label, tokenizer, max_length)
dev_dataset = ClassificationDataset(dev_data, dev_label, tokenizer, max_length)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)

In [10]:
test_dataset = ClassificationDataset(test_data, test_label, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Model Preparation

In [11]:
num_classes = 2

In [12]:
class RobertaClassifier(nn.Module):
    def __init__(self, roberta_model, num_classes, dropout=0.1):
        super(RobertaClassifier, self).__init__()
        self.roberta = roberta_model
        self.dropout = nn.Dropout(dropout)
        self.ffn = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.ffn(x)
        return logits

In [13]:
roberta_model = RobertaModel.from_pretrained('roberta-base')

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
model = RobertaClassifier(roberta_model, num_classes)

In [15]:
learning_rate = 2e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

RobertaClassifier(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from tqdm import tqdm

def calculate_metrics(y_true, y_pred, y_pred_proba, num_classes):
    if num_classes == 2:
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred),
            'recall': recall_score(y_true, y_pred),
            'f1': f1_score(y_true, y_pred)
        }
    else:
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred, average='weighted'),
            'recall': recall_score(y_true, y_pred, average='weighted'),
            'f1': f1_score(y_true, y_pred, average='weighted')
        }
    
    # Calculate ROC-AUC score
    if num_classes == 2:
        metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba[:, 1])
    else:
        try:
            metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average='weighted')
        except:
            metrics['roc_auc'] = None
            
    return metrics

In [17]:
def evaluate(model, data_loader, device, num_classes):
    model.eval()
    all_labels = []
    all_predictions = []
    all_predictions_proba = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            all_predictions_proba.extend(probabilities.cpu().numpy())

    all_labels = np.array(all_labels)
    all_predictions = np.array(all_predictions)
    all_predictions_proba = np.array(all_predictions_proba)

    return calculate_metrics(all_labels, all_predictions, all_predictions_proba, num_classes)

In [18]:
def train(model, train_loader, val_loader, criterion, optimizer, device, num_epochs):
    best_val_metrics = {'f1': 0.0}
    history = {'train': [], 'val': []}

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        all_train_labels = []
        all_train_predictions = []
        all_train_predictions_proba = []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_train_labels.extend(labels.cpu().numpy())
            all_train_predictions.extend(predicted.cpu().numpy())
            all_train_predictions_proba.extend(probabilities.cpu().detach().numpy())

        # Calculate training metrics
        all_train_labels = np.array(all_train_labels)
        all_train_predictions = np.array(all_train_predictions)
        all_train_predictions_proba = np.array(all_train_predictions_proba)
        train_metrics = calculate_metrics(all_train_labels, all_train_predictions, 
                                       all_train_predictions_proba, num_classes)
        
        # Validation phase
        val_metrics = evaluate(model, val_loader, device, num_classes)
        
        # Store metrics history
        history['train'].append({
            'loss': train_loss / len(train_loader),
            **train_metrics
        })
        history['val'].append(val_metrics)

        # Print epoch results
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("Training Metrics:")
        print(f"Loss: {train_loss/len(train_loader):.4f}")
        for metric, value in train_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
        
        print("\nValidation Metrics:")
        for metric, value in val_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")

        # Save best model based on validation F1 score
        if val_metrics['f1'] > best_val_metrics['f1']:
            best_val_metrics = val_metrics
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'metrics': val_metrics,
            }, 'best_model.pth')

    print("\nBest Validation Metrics:")
    for metric, value in best_val_metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")
    
    return history

In [19]:
num_epochs = 5

optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
history = train(model, train_loader, dev_loader, criterion, optimizer, device, num_epochs)

Epoch 1/5: 100%|██████████| 141/141 [00:54<00:00,  2.56it/s]



Epoch 1/5
Training Metrics:
Loss: 0.3153
Accuracy: 0.8551
Precision: 0.8397
Recall: 0.8641
F1: 0.8517
Roc_auc: 0.9417

Validation Metrics:
Accuracy: 0.9564
Precision: 0.9975
Recall: 0.9062
F1: 0.9496
Roc_auc: 0.9792


Epoch 2/5: 100%|██████████| 141/141 [00:53<00:00,  2.64it/s]



Epoch 2/5
Training Metrics:
Loss: 0.1004
Accuracy: 0.9673
Precision: 0.9702
Recall: 0.9616
F1: 0.9659
Roc_auc: 0.9931

Validation Metrics:
Accuracy: 0.9699
Precision: 0.9951
Recall: 0.9382
F1: 0.9658
Roc_auc: 0.9959


Epoch 3/5: 100%|██████████| 141/141 [00:53<00:00,  2.65it/s]



Epoch 3/5
Training Metrics:
Loss: 0.0501
Accuracy: 0.9835
Precision: 0.9825
Recall: 0.9834
F1: 0.9829
Roc_auc: 0.9979

Validation Metrics:
Accuracy: 0.9751
Precision: 0.9814
Recall: 0.9634
F1: 0.9723
Roc_auc: 0.9964


Epoch 4/5: 100%|██████████| 141/141 [00:53<00:00,  2.64it/s]



Epoch 4/5
Training Metrics:
Loss: 0.0314
Accuracy: 0.9904
Precision: 0.9917
Recall: 0.9884
F1: 0.9900
Roc_auc: 0.9987

Validation Metrics:
Accuracy: 0.9730
Precision: 0.9768
Recall: 0.9634
F1: 0.9700
Roc_auc: 0.9971


Epoch 5/5: 100%|██████████| 141/141 [00:53<00:00,  2.64it/s]



Epoch 5/5
Training Metrics:
Loss: 0.0157
Accuracy: 0.9955
Precision: 0.9949
Recall: 0.9958
F1: 0.9954
Roc_auc: 0.9997

Validation Metrics:
Accuracy: 0.9709
Precision: 1.0000
Recall: 0.9359
F1: 0.9669
Roc_auc: 0.9976

Best Validation Metrics:
Accuracy: 0.9751
Precision: 0.9814
Recall: 0.9634
F1: 0.9723
Roc_auc: 0.9964


In [21]:
import pandas as pd

pd.DataFrame(history['train'])

,loss,accuracy,precision,recall,f1,roc_auc
0,0.315275,0.855140,0.839695,0.864140,0.851742,0.941657
1,0.100371,0.967290,0.970163,0.961645,0.965885,0.993050
2,0.050104,0.983534,0.982456,0.983364,0.982910,0.997895
3,0.031426,0.990432,0.991655,0.988447,0.990049,0.998695
4,0.015688,0.995550,0.994922,0.995841,0.995381,0.999681


In [22]:
pd.DataFrame(history['val'])

,accuracy,precision,recall,f1,roc_auc
0,0.956386,0.997481,0.906178,0.949640,0.979161
1,0.969886,0.995146,0.938215,0.965842,0.995932
2,0.975078,0.981352,0.963387,0.972286,0.996420
3,0.973001,0.976798,0.963387,0.970046,0.997098
4,0.970924,1.000000,0.935927,0.966903,0.997599


In [23]:
evaluate(model, test_loader, device, num_classes)

{'accuracy': 0.9626168224299065,
 'precision': 0.9976470588235294,
 'recall': 0.9237472766884531,
 'f1': 0.9592760180995475,
 'roc_auc': 0.9982060725524777}